Продолжаем предобрабатывать данные

В этом уроке познакомимся с merge и join в Pandas
А также рассмотрим продвинутый прием по работе с данными при помощи метода apply

http://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
submissions_data = pd.read_csv('submissions_data_train.csv');
submissions_data.head()

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


In [3]:
submissions_data['date'] = pd.to_datetime(submissions_data.timestamp, unit='s')
submissions_data['day'] = submissions_data['date'].dt.date
submissions_data.head()

,step_id,timestamp,submission_status,user_id,date,day
0,31971,1434349275,correct,15853,2015-06-15 06:21:15,2015-06-15
1,31972,1434348300,correct,15853,2015-06-15 06:05:00,2015-06-15
2,31972,1478852149,wrong,15853,2016-11-11 08:15:49,2016-11-11
3,31972,1478852164,correct,15853,2016-11-11 08:16:04,2016-11-11
4,31976,1434348123,wrong,15853,2015-06-15 06:02:03,2015-06-15


In [4]:
users_scores = submissions_data.pivot_table(index='user_id',
                            columns='submission_status',
                            values='step_id',
                            aggfunc='count',
                            fill_value=0).reset_index()
users_scores.head()

submission_status,user_id,correct,wrong
0,2,2,0
1,3,29,23
2,5,2,2
3,8,9,21
4,14,0,1


In [5]:
event_data = pd.read_csv('event_data_train.csv')
event_data['date'] = pd.to_datetime(event_data.timestamp, unit='s')
event_data['day'] = event_data.date.dt.date

In [6]:
# unique days when particular user was online
by_user_ts = event_data[['user_id', 'day', 'timestamp']].\
    drop_duplicates(subset=['user_id', 'day']).\
    groupby('user_id').\
    timestamp.apply(list)
by_user_ts.head()

user_id
1                                         [1472827464]
2                             [1514383364, 1519226966]
3    [1434358476, 1441257725, 1441440209, 144153391...
5                             [1466156809, 1499859621]
7                                         [1521634660]
Name: timestamp, dtype: object

In [7]:
gap_data = by_user_ts.apply(np.diff);
gap_data.head()

user_id
1                                                  []
2                                           [4843602]
3    [6899249, 182484, 93710, 2768870, 171400, 78712]
5                                          [33702812]
7                                                  []
Name: timestamp, dtype: object

In [8]:
gap_data.values

array([array([], dtype=int32), array([4843602]),
       array([6899249,  182484,   93710, 2768870,  171400,   78712]), ...,
       array([   86128,   136759,    94899,  1422583, 14347289,   505061,
          98252,    57019]),
       array([1567990]), array([], dtype=int32)], dtype=object)

In [9]:
c = np.concatenate(gap_data.values, axis=0)

In [10]:
gap_data = pd.Series(c)
#(gap_data / (24*60*60)).head()
# diff in days

In [11]:
event_data[event_data.timestamp == event_data.timestamp.min()]

,step_id,timestamp,action,user_id,date,day
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15


In [12]:
#submissions_data.timestamp.min()
submissions_data[submissions_data.timestamp == submissions_data.timestamp.min()]

,step_id,timestamp,submission_status,user_id,date,day
37939,32929,1434340944,correct,17632,2015-06-15 04:02:24,2015-06-15


In [13]:
#submissions_data.groupby('user_id').agg({''})
users_scores[users_scores.correct == users_scores.correct.max()]

submission_status,user_id,correct,wrong
439,1046,421,513


In [14]:
user_scores = submissions_data.pivot_table(index='user_id',
                                        columns='submission_status',
                                        values='step_id',
                                        aggfunc='count',
                                        fill_value=0).reset_index()
user_scores.sort_values(by='correct', ascending=False).head()

submission_status,user_id,correct,wrong
439,1046,421,513
2099,5552,186,93
6415,17135,168,177
2344,6183,142,65
3208,8461,135,42


In [15]:
#Пользователь, который провел на курсе больше всего дней 
event_data.groupby('user_id')['day'].nunique().idxmax()

1046

In [18]:
#submissions_data[submissions_data.submission_status == 'correct'].groupby('step_id').agg({'submission_status': 'count'}).sort_values(by=['submission_status'], ascending=False).head(20)

In [19]:
gap_data.quantile(0.90) / (24*60*60)

18.325995370370403

In [34]:
users_data = event_data.groupby('user_id', as_index=False).agg({'timestamp':'max'}).\
    rename(index=str, columns={'timestamp':'last_timestamp'})

In [35]:
drop_out_threshold = 30*24*60*60
now = event_data.timestamp.max()
users_data['is_gone_user'] = (now - users_data.last_timestamp > drop_out_threshold)

In [36]:
users_data.head()

,user_id,last_timestamp,is_gone_user
0,1,1472827464,True
1,2,1519226966,True
2,3,1444581588,True
3,5,1499859939,True
4,7,1521634660,True


In [37]:
users_data.is_gone_user.value_counts()

True     17844
False     1390
Name: is_gone_user, dtype: int64

In [38]:
users_data.head()

,user_id,last_timestamp,is_gone_user
0,1,1472827464,True
1,2,1519226966,True
2,3,1444581588,True
3,5,1499859939,True
4,7,1521634660,True


In [39]:
users_scores.head()

submission_status,user_id,correct,wrong
0,2,2,0
1,3,29,23
2,5,2,2
3,8,9,21
4,14,0,1


In [40]:
users_data = users_data.merge(user_scores, on='user_id', how='outer').head()

In [41]:
users_data = users_data.fillna(0)
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong
0,1,1472827464,True,0.0,0.0
1,2,1519226966,True,2.0,0.0
2,3,1444581588,True,29.0,23.0
3,5,1499859939,True,2.0,2.0
4,7,1521634660,True,0.0,0.0


In [42]:
users_events_data = event_data.pivot_table(index='user_id',
                                          columns='action',
                                          values='step_id',
                                          aggfunc='count',
                                          fill_value=0).reset_index()
users_events_data.head()

action,user_id,discovered,passed,started_attempt,viewed
0,1,1,0,0,1
1,2,9,9,2,10
2,3,91,87,30,192
3,5,11,11,4,12
4,7,1,1,0,1


In [43]:
users_data = users_data.merge(users_events_data, on='user_id', how='outer')
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed
0,1,1.472827e+09,True,0.0,0.0,1,0,0,1
1,2,1.519227e+09,True,2.0,0.0,9,9,2,10
2,3,1.444582e+09,True,29.0,23.0,91,87,30,192
3,5,1.499860e+09,True,2.0,2.0,11,11,4,12
4,7,1.521635e+09,True,0.0,0.0,1,1,0,1


In [45]:
users_data.dtypes

user_id              int64
last_timestamp     float64
is_gone_user        object
correct            float64
wrong              float64
discovered           int64
passed               int64
started_attempt      int64
viewed               int64
dtype: object

In [48]:
users_days = event_data.groupby('user_id').day.nunique().to_frame().reset_index()

In [51]:
users_data = users_data.merge(users_days, on='user_id', how='outer')

In [53]:
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day_x,day_y
0,1,1.472827e+09,True,0.0,0.0,1,0,0,1,1,1
1,2,1.519227e+09,True,2.0,0.0,9,9,2,10,2,2
2,3,1.444582e+09,True,29.0,23.0,91,87,30,192,7,7
3,5,1.499860e+09,True,2.0,2.0,11,11,4,12,2,2
4,7,1.521635e+09,True,0.0,0.0,1,1,0,1,1,1


In [55]:
users_data.user_id.nunique()

19234

In [57]:
users_data['passed_course'] = users_data.passed > 170

In [59]:
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day_x,day_y,passed_course
0,1,1.472827e+09,True,0.0,0.0,1,0,0,1,1,1,False
1,2,1.519227e+09,True,2.0,0.0,9,9,2,10,2,2,False
2,3,1.444582e+09,True,29.0,23.0,91,87,30,192,7,7,False
3,5,1.499860e+09,True,2.0,2.0,11,11,4,12,2,2,False
4,7,1.521635e+09,True,0.0,0.0,1,1,0,1,1,1,False


In [61]:
users_data.passed_course.value_counts()

False    17809
True      1425
Name: passed_course, dtype: int64

In [63]:
users_data.groupby('passed_course').count()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day_x,day_y
passed_course,,,,,,,,,,,
False,17809,5,5,5,5,17809,17809,17809,17809,17809,17809
True,1425,0,0,0,0,1425,1425,1425,1425,1425,1425


In [72]:
tmp = users_data.groupby('passed_course').count()
#(tmp.user_id[True] / event_data.user_id.nunique()) * 100
(tmp.user_id[True] / tmp.user_id[False]) * 100

8.001572238755687